### Creating a Complete ETL Pipeline using Delta Live Tables (DLT)

In [ ]:
dbutils.fs.cp("file:/Workspace/Shared/orders.csv", "dbfs:/FileStore/streaming/input/orders.csv") 

True

Task 1: Create an ETL Pipeline using DLT (Python)

In [ ]:
import dlt
from pyspark.sql.functions import col

@dlt.table
def transformed_orders():
    # Read CSV source
    df = spark.read.csv("dbfs:/FileStore/streaming/input/orders.csv", header=True)
    
    # Transform data
    df_transformed = df.withColumn("TotalAmount", col("Quantity") * col("Price")) \
                       .filter(col("Quantity") > 1)
    
    return df_transformed

@dlt.table
def incremental_orders():
    return dlt.read_stream("transformed_orders")

Name,Type
OrderID,string
OrderDate,string
CustomerID,string
Product,string
Quantity,string
Price,string
TotalAmount,double


Task 2: Create an ETL Pipeline using DLT (SQL)

In [ ]:
# Load the CSV file
df_orders = spark.read.format("csv").option("header", "true").option("inferSchema", "true").load("dbfs:/FileStore/streaming/input/orders.csv")

df_orders.write.format("delta").mode("overwrite").save("/delta/orders")


In [ ]:
%sql
CREATE OR REPLACE LIVE TABLE transformed_orders AS
SELECT *, Quantity * Price AS TotalAmount
FROM delta.`/delta/orders`
WHERE Quantity > 1;

Name,Type
OrderID,int
OrderDate,date
CustomerID,string
Product,string
Quantity,int
Price,int
TotalAmount,int


Task 3: Perform Read, Write, Update, and Delete Operations on Delta Table (SQL + PySpark)

1. Read the Data from the Delta Table

In [ ]:
# Reading the Delta table in PySpark
df = spark.read.format("delta").load("/delta/orders")
df.show()

+-------+----------+----------+-------+--------+-----+
|OrderID| OrderDate|CustomerID|Product|Quantity|Price|
+-------+----------+----------+-------+--------+-----+
|    101|2024-01-01|      C001| Laptop|       2| 1000|
|    102|2024-01-02|      C002|  Phone|       1|  500|
|    103|2024-01-03|      C003| Tablet|       3|  300|
|    104|2024-01-04|      C004|Monitor|       1|  150|
|    105|2024-01-05|      C005|  Mouse|       5|   20|
+-------+----------+----------+-------+--------+-----+



In [ ]:
%sql
-- Reading the Delta table using SQL
SELECT * FROM delta.`/delta/orders`;

OrderID,OrderDate,CustomerID,Product,Quantity,Price
101,2024-01-01,C001,Laptop,2,1000
102,2024-01-02,C002,Phone,1,500
103,2024-01-03,C003,Tablet,3,300
104,2024-01-04,C004,Monitor,1,150
105,2024-01-05,C005,Mouse,5,20


2. Update the Price of a Product (e.g., increase Laptop price by 10%)

In [ ]:
from delta.tables import DeltaTable
from pyspark.sql.functions import expr

# Load the Delta table
delta_table = DeltaTable.forPath(spark, "/delta/orders")

# Update the Price of 'Laptop' by increasing it by 10%
delta_table.update(
    condition = expr("Product = 'Laptop'"),
    set = { "Price": expr("Price * 1.1") }
)


In [ ]:
%sql
-- Update product price using SQL (increase Laptop price by 10%)
UPDATE delta.`/delta/orders`
SET Price = Price * 1.1
WHERE Product = 'Laptop';


num_affected_rows
1


3. Delete Rows where Quantity is Less Than 2

In [ ]:
# Delete rows where Quantity is less than 2
delta_table.delete(condition = expr("Quantity < 2"))

In [ ]:
%sql
-- Delete rows where Quantity is less than 2
DELETE FROM delta.`/delta/orders`
WHERE Quantity < 2;

num_affected_rows
0


4. Insert a New Record into the Delta Table

In [ ]:
from pyspark.sql.types import IntegerType, StringType, StructField, StructType

# Insert a new record into the Delta table
new_data = [(106, 'C006', 'Keyboard', 2, 50)]
columns = ["OrderID", "CustomerID", "Product", "Quantity", "Price"]

# Define the schema of the Delta table
schema = StructType([
    StructField("OrderID", IntegerType(), nullable=False),
    StructField("CustomerID", StringType(), nullable=False),
    StructField("Product", StringType(), nullable=False),
    StructField("Quantity", IntegerType(), nullable=False),
    StructField("Price", IntegerType(), nullable=False)
])

# Create a DataFrame with new data and specified schema
new_df = spark.createDataFrame(new_data, schema)

# Append the new data to the Delta table
new_df.write.format("delta").mode("append").save("/delta/orders")

In [ ]:
%sql
-- Insert a new record using SQL
INSERT INTO delta.`/delta/orders`
VALUES (107, '2024-01-06', 'C006', 'Keyboard', 2, 50);


num_affected_rows,num_inserted_rows
1,1


Task 4: Merge Data (Slowly Changing Dimension - SCD Type 2)

In [ ]:
# New data representing updated orders
new_data = [(101, '2024-01-10', 'C001', 'Laptop', 2, 1200),  # Updated order for Laptop
            (106, '2024-01-12', 'C006', 'Keyboard', 3, 50)]   # New order for Keyboard

columns = ["OrderID", "OrderDate", "CustomerID", "Product", "Quantity", "Price"]

# Create a DataFrame with the new data
new_df = spark.createDataFrame(new_data, columns)

# Merge the new data into the Delta table (SCD Type 2)
delta_table.alias("old").merge(
    new_df.alias("new"),
    "old.OrderID = new.OrderID"
).whenMatchedUpdate(set={
    "OrderDate": "new.OrderDate",
    "CustomerID": "new.CustomerID",
    "Product": "new.Product",
    "Quantity": "new.Quantity",
    "Price": "new.Price"
}).whenNotMatchedInsertAll().execute()

Task 5: Explore Delta Table Internals

In [ ]:
# Check the transaction history using PySpark
delta_table.history().show()
# Check file details using PySpark
spark.sql("DESCRIBE DETAIL delta.`/delta/orders`").show()

+-------+-------------------+----------------+--------------------+---------+--------------------+----+-----------------+--------------------+-----------+-----------------+-------------+--------------------+------------+--------------------+
|version|          timestamp|          userId|            userName|operation| operationParameters| job|         notebook|           clusterId|readVersion|   isolationLevel|isBlindAppend|    operationMetrics|userMetadata|          engineInfo|
+-------+-------------------+----------------+--------------------+---------+--------------------+----+-----------------+--------------------+-----------+-----------------+-------------+--------------------+------------+--------------------+
|     35|2024-09-17 05:04:39|2929584751483774|azuser2141_mml.lo...| OPTIMIZE|{predicate -> [],...|NULL|{418182220691607}|0912-053226-p5usobai|         34|SnapshotIsolation|        false|{numRemovedFiles ...|        NULL|Databricks-Runtim...|
|     34|2024-09-17 05:04:37|292

In [ ]:
%sql
-- Check the transaction history using SQL
DESCRIBE HISTORY delta.`/delta/orders`;
-- Check file details using SQL
DESCRIBE DETAIL delta.`/delta/orders`;

format,id,name,description,location,createdAt,lastModified,partitionColumns,clusteringColumns,numFiles,sizeInBytes,properties,minReaderVersion,minWriterVersion,tableFeatures,statistics
delta,505a9da5-3e4f-469c-ba9c-1974afaed544,null,null,dbfs:/delta/orders,2024-09-17T04:41:18.952Z,2024-09-17T05:04:39Z,List(),List(),1,1594,Map(delta.enableDeletionVectors -> true),3,7,List(deletionVectors),"Map(numRowsDeletedByDeletionVectors -> 0, numDeletionVectors -> 0)"


Task 6: Time Travel in Delta Tables

In [ ]:
# Query the table as it existed at a specific version
df_version = spark.read.format("delta").option("versionAsOf", 1).load("/delta/orders")
df_version.show()

# Query the table as it existed at a specific timestamp
df_timestamp = spark.read.format("delta").option("timestampAsOf", "2024-09-17T04:41:20Z").load("/delta/orders")
df_timestamp.show()

+-------+----------+----------+-------+--------+-----+
|OrderID| OrderDate|CustomerID|Product|Quantity|Price|
+-------+----------+----------+-------+--------+-----+
|    102|2024-01-02|      C002|  Phone|       1|  500|
|    103|2024-01-03|      C003| Tablet|       3|  300|
|    104|2024-01-04|      C004|Monitor|       1|  150|
|    105|2024-01-05|      C005|  Mouse|       5|   20|
|    101|2024-01-01|      C001| Laptop|       2| 1100|
+-------+----------+----------+-------+--------+-----+

+-------+----------+----------+-------+--------+-----+
|OrderID| OrderDate|CustomerID|Product|Quantity|Price|
+-------+----------+----------+-------+--------+-----+
|    101|2024-01-01|      C001| Laptop|       2| 1000|
|    102|2024-01-02|      C002|  Phone|       1|  500|
|    103|2024-01-03|      C003| Tablet|       3|  300|
|    104|2024-01-04|      C004|Monitor|       1|  150|
|    105|2024-01-05|      C005|  Mouse|       5|   20|
+-------+----------+----------+-------+--------+-----+



Task 7: Optimize Delta Table

In [ ]:
# Optimize the table and Z-order by Product column
spark.sql("OPTIMIZE delta.`/delta/orders` ZORDER BY (Product)")
# Vacuum the Delta table to remove old versions of the files
spark.sql("VACUUM delta.`/delta/orders` RETAIN 168 HOURS")


DataFrame[path: string]

Task 8: Converting Parquet Files to Delta Format

In [ ]:
from pyspark.sql.types import StructType, StructField, StringType, IntegerType, DoubleType, DateType

# Define the schema of your Parquet file
schema = StructType([
    StructField("OrderID", StringType(), True),
    StructField("OrderDate", DateType(), True),
    StructField("CustomerID", StringType(), True),
    StructField("Product", StringType(), True),
    StructField("Quantity", IntegerType(), True),
    StructField("Price", DoubleType(), True),
])

parquet_path = "/dbfs/FileStore/"
delta_path = "/dbfs/FileStore/delta_table"

# Read Parquet data with the defined schema
df_parquet = spark.read.format("parquet").schema(schema).load(parquet_path)

# Write the data in Delta format
df_parquet.write.format("delta").mode("overwrite").save(delta_path)

# Register the Delta table
spark.sql(f"CREATE TABLE delta_table USING DELTA LOCATION '{delta_path}'")

# Query the newly converted Delta table
df_converted = spark.sql("SELECT * FROM delta_table")
df_converted.show()


+-------+---------+----------+-------+--------+-----+
|OrderID|OrderDate|CustomerID|Product|Quantity|Price|
+-------+---------+----------+-------+--------+-----+
+-------+---------+----------+-------+--------+-----+

